## CREST XD DATA CLEANING


In [1]:
#required libraries
import os #for reading files
import pandas as pd #for data manipulation (cleaning the dataset,checking for nulls,dropping nulls, checking for duplicates etc)
import re #regular expressions, which allow you to search, match, and manipulate text by defining patterns.
from textblob import TextBlob #A sentiment analysis package
from nltk.sentiment.vader import SentimentIntensityAnalyzer #Another sentiment analysis tool
import nltk #For the sentiment analysis

In [4]:

# Download the VADER lexicon
nltk.download('vader_lexicon') #a pre-trained model for sentiment analysis.NB: It takes into consideration the emojis in the text 
#and 1=positive,0=Neutral,-1=Negative 

[nltk_data] Downloading package vader_lexicon to C:\Users\Olaniyi
[nltk_data]     David\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
#The directory where my csv file is stored
os.chdir(r"c:\\Users\\Olaniyi David\\Desktop")


In [3]:
#reading the csv file using pandas
data = pd.read_csv("web3_tweets.csv")

In [4]:
#Displaying the first five columns in the dataset
data.head()

,Tweet_count,Username,Text,Created_At,Retweets,Likes,Views
0,1,JackTheRippler ©️,🚨PAY ATTENTION! A VIDEO REEMERGES OF GARY GENS...,Thu Oct 24 08:57:45 +0000 2024,1,8,224.0
1,2,Patrick 🔺️9000,Flash Giveaway 🚨🚨\n\nGiving 3 WL spots to next...,Thu Oct 24 08:33:04 +0000 2024,16,18,238.0
2,3,BeL2,Bitcoin is the cornerstone of the next financi...,Thu Oct 24 07:03:04 +0000 2024,7,21,293.0
3,4,🕹 𝐏𝟐𝐄_𝐒𝐏𝐀𝐂𝐄,🎮 Exciting news! @RavegameNFT has teamed up wi...,Tue Oct 22 14:35:17 +0000 2024,48,76,2755.0
4,5,🔮Metaverse Worlds,🎯 @KingdomXGame is excited to announce its par...,Tue Oct 22 13:23:17 +0000 2024,43,71,6674.0


In [5]:
#Numbr of rows and columns contained in the dataset 
data.shape

(3462, 7)

In [6]:
#Checking for null values in each column
data.isnull().sum()

Tweet_count     0
Username        0
Text            0
Created_At      0
Retweets        0
Likes           0
Views          15
dtype: int64

## Creating new attributes

1. Retention rate = The sum of the retweets and likes divided by the total numbr of views. NB: 0 = bad retention, 2=Excellent retention
2. Total Number of engagments
3. From the the created_at column, these columns were created (Day, Hour and Time of the day(Morning, evening,afternoon etc)
4. Keyword columnn was created by mapping each tweet to the respective keywords used in scraping them
5. Two extra columns which contained the sentiment analysis results for two different models (Textblob and Vader) 

In [7]:
data['retention_rate'] = (data['Retweets'] + data['Likes']) / data['Views']

In [8]:
data['total_engagements'] = data['Retweets'] + data['Likes']

In [9]:
# Extract the day abbreviation (first 3 characters)
data['day'] = data['Created_At'].str[:3]

# Mapping of day abbreviations to full weekday names
day_map = {
    'Mon': 'Monday',
    'Tue': 'Tuesday',
    'Wed': 'Wednesday',
    'Thu': 'Thursday',
    'Fri': 'Friday',
    'Sat': 'Saturday',
    'Sun': 'Sunday'
}

# Map the abbreviations to full weekday names
data['day'] = data['day'].map(day_map)


In [10]:
# Extract the time portion (using split to get the part with the hour)
data['hour'] = data['Created_At'].str.split(' ').str[3]

# Extract the hour from the time string
data['hour'] = data['hour'].str.split(':').str[0].astype(int)


In [11]:
# Define a function to categorize the time of day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

# Apply the function to create a new column with the time of day
data['time_of_day'] = data['hour'].apply(categorize_time_of_day)

## Text preprocessing before extracting keywords

In [12]:
#convert all text to lowercase
data['Text'] = data['Text'].str.lower()

In [13]:
#replace the\n in all text with''(an empty string)
data['Text'] = data['Text'].str.replace('\n', ' ')


In [14]:

#Removed the hashtag, @ symbol and links to elimante noise and to avoid confusing the model
def clean_tweet(text):
    text = text.str.replace(r'@\w+', '', regex=True)  # Remove @mentions
    text = text.str.replace(r'#\w+', '', regex=True)  # Remove hashtags
    text = text.str.replace(r'http\S+|www\S+|https\S+', '', regex=True)  # Remove URLs
    
    return text

# Apply the function to the 'tweet' column
data['Text'] = clean_tweet(data['Text'])

In [15]:
#List of all the keywords and extracting them to create a new column
keywords = ["defi", "nfts", "daos", "metaverse", "layer 2", "gaming", 
    "bridges", "p2e", "wallets", "staking", "liquidity", 
    "airdrops", "stablecoins", "assets", "guilds", "shilling", 
    "farming", "socialfi", "tokenomics", "interoperability","pos","proof of stake"]

# Create a regex pattern for whole words, case-insensitive
pattern = '|'.join(rf'\b{re.escape(kw)}\b' for kw in keywords)

# Create a new column to show the keywords found
data['keyword'] = data['Text'].str.findall(pattern, flags=re.IGNORECASE).apply(lambda x: ', '.join(set(x)))

In [16]:
#It was observed that some rows contained empty strings so i replaced them with NA which is easier to work with
data['keyword'] = data['keyword'].replace('', pd.NA)

In [17]:
data['keyword'] = data['keyword'].replace(pd.NA,"general")

In [18]:
#defining the function that carries out the sentiment analysis
def get_sentiment_textblob(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity
def get_sentiment_vader(text):
    sid = SentimentIntensityAnalyzer()
    score = sid.polarity_scores(text)
    return score['compound']


In [19]:
# Using TextBlob
data['sentiment_textblob'] = data['Text'].apply(get_sentiment_textblob)

# Using VADER
data['sentiment_vader'] = data['Text'].apply(get_sentiment_vader)

In [20]:
data.head(50)

,Tweet_count,Username,Text,Created_At,Retweets,Likes,Views,retention_rate,total_engagements,day,hour,time_of_day,keyword,sentiment_textblob,sentiment_vader
0,1,JackTheRippler ©️,🚨pay attention! a video reemerges of gary gens...,Thu Oct 24 08:57:45 +0000 2024,1,8,224.0,0.040179,9,Thursday,8,Morning,"defi, liquidity",0.241364,0.6562
1,2,Patrick 🔺️9000,flash giveaway 🚨🚨 giving 3 wl spots to next m...,Thu Oct 24 08:33:04 +0000 2024,16,18,238.0,0.142857,34,Thursday,8,Morning,general,0.000000,0.7906
2,3,BeL2,bitcoin is the cornerstone of the next financi...,Thu Oct 24 07:03:04 +0000 2024,7,21,293.0,0.095563,28,Thursday,7,Morning,general,0.081169,0.5574
3,4,🕹 𝐏𝟐𝐄_𝐒𝐏𝐀𝐂𝐄,🎮 exciting news! has teamed up with to bring...,Tue Oct 22 14:35:17 +0000 2024,48,76,2755.0,0.045009,124,Tuesday,14,Afternoon,metaverse,0.092969,0.6219
4,5,🔮Metaverse Worlds,🎯 is excited to announce its partnership with...,Tue Oct 22 13:23:17 +0000 2024,43,71,6674.0,0.017081,114,Tuesday,13,Afternoon,layer 2,0.187500,0.7717
5,6,XPLA (ⓧ.ⓧ),🥇introducing the 1st palympic!🥇 how to partic...,Thu Oct 24 07:47:11 +0000 2024,2,10,361.0,0.033241,12,Thursday,7,Morning,nfts,0.350000,0.8715
6,7,🔮Metaverse Worlds,🎮 partners with to offer exclusive rewards t...,Wed Oct 23 17:39:10 +0000 2024,52,75,6447.0,0.019699,127,Wednesday,17,Evening,gaming,0.600000,0.7506
7,8,DeFi Tycoon,exciting news! is launching a mine to earn $m...,Thu Oct 24 07:46:38 +0000 2024,7,22,244.0,0.118852,29,Thursday,7,Morning,staking,0.375000,0.9080
8,9,🔮Metaverse Worlds,🔮 daily digest 10/22/2024 🔮 partnership wit...,Wed Oct 23 09:31:00 +0000 2024,49,74,17496.0,0.007030,123,Wednesday,9,Morning,layer 2,0.000000,0.4215
9,10,Hanzo ㊗️,"to make $1m, you need one 1000x token. and ev...",Sat Oct 19 17:44:46 +0000 2024,322,1775,334820.0,0.006263,2097,Saturday,17,Evening,liquidity,0.250000,0.0000


In [21]:
#New size of the dataset after 
data.shape

(3462, 15)

In [23]:
data.isnull().sum()

Tweet_count            0
Username               0
Text                   0
Created_At             0
Retweets               0
Likes                  0
Views                 15
retention_rate        15
total_engagements      0
day                    0
hour                   0
time_of_day            0
keyword                0
sentiment_textblob     0
sentiment_vader        0
dtype: int64

In [25]:
data.head(50)

,Tweet_count,Username,Text,Created_At,Retweets,Likes,Views,retention_rate,total_engagements,day,hour,time_of_day,keyword,sentiment_textblob,sentiment_vader
0,1,JackTheRippler ©️,🚨pay attention! a video reemerges of gary gens...,Thu Oct 24 08:57:45 +0000 2024,1,8,224.0,0.040179,9,Thursday,8,Morning,"liquidity, defi",0.241364,0.6562
1,2,Patrick 🔺️9000,flash giveaway 🚨🚨 giving 3 wl spots to next m...,Thu Oct 24 08:33:04 +0000 2024,16,18,238.0,0.142857,34,Thursday,8,Morning,general,0.000000,0.7906
2,3,BeL2,bitcoin is the cornerstone of the next financi...,Thu Oct 24 07:03:04 +0000 2024,7,21,293.0,0.095563,28,Thursday,7,Morning,general,0.081169,0.5574
3,4,🕹 𝐏𝟐𝐄_𝐒𝐏𝐀𝐂𝐄,🎮 exciting news! has teamed up with to bring...,Tue Oct 22 14:35:17 +0000 2024,48,76,2755.0,0.045009,124,Tuesday,14,Afternoon,metaverse,0.092969,0.6219
4,5,🔮Metaverse Worlds,🎯 is excited to announce its partnership with...,Tue Oct 22 13:23:17 +0000 2024,43,71,6674.0,0.017081,114,Tuesday,13,Afternoon,layer 2,0.187500,0.7717
5,6,XPLA (ⓧ.ⓧ),🥇introducing the 1st palympic!🥇 how to partic...,Thu Oct 24 07:47:11 +0000 2024,2,10,361.0,0.033241,12,Thursday,7,Morning,nfts,0.350000,0.8715
6,7,🔮Metaverse Worlds,🎮 partners with to offer exclusive rewards t...,Wed Oct 23 17:39:10 +0000 2024,52,75,6447.0,0.019699,127,Wednesday,17,Evening,gaming,0.600000,0.7506
7,8,DeFi Tycoon,exciting news! is launching a mine to earn $m...,Thu Oct 24 07:46:38 +0000 2024,7,22,244.0,0.118852,29,Thursday,7,Morning,staking,0.375000,0.9080
8,9,🔮Metaverse Worlds,🔮 daily digest 10/22/2024 🔮 partnership wit...,Wed Oct 23 09:31:00 +0000 2024,49,74,17496.0,0.007030,123,Wednesday,9,Morning,layer 2,0.000000,0.4215
9,10,Hanzo ㊗️,"to make $1m, you need one 1000x token. and ev...",Sat Oct 19 17:44:46 +0000 2024,322,1775,334820.0,0.006263,2097,Saturday,17,Evening,liquidity,0.250000,0.0000


In [25]:
data.dtypes

Tweet_count                         int64
Username                           object
Text                               object
Created_At                         object
Retweets                            int64
Likes                               int64
Views                             float64
retention_rate                    float64
total_engagements                   int64
keyword                            object
sentiment_textblob                float64
sentiment_vader                   float64
datetime              datetime64[ns, UTC]
dtype: object

In [46]:
data.loc[2196, 'Text']

'👂 hear from  , the co-founder of , on the real challenges of bringing  and  together in a world led by tech giants like  &amp; ! 🚀  🔥catch the full interview here  '

## Twitter Script

In [ ]:
import asyncio
import csv
import os
import random
import pyodbc
from datetime import datetime
from twikit import Client, TooManyRequests


# Credentials
USERNAME = 'williamms94'
PASSWORD = 'Fifehanmi100.@'

# Initialize client
client = Client('en-US')

# Search query for Web3
WEB3_TOPICS = [
    "DeFi", "NFTs", "DAOs", "Metaverse", "Layer 2", "Gaming", 
    "Bridges", "P2E", "Wallets", "Staking", "Liquidity", 
    "Airdrops", "Stablecoins", "Assets", "Guilds", "Shilling", 
    "Farming", "SocialFi", "Tokenomics", "Interoperability"
]

# Combine topics into a single search query with "OR"
QUERY = ' OR '.join(WEB3_TOPICS)

# Delay between requests to avoid rate limits
DELAY_SECONDS = (2, 5)
RATE_LIMIT_DELAY = 900  # 15 minutes

CSV_FILENAME = 'web3_tweets.csv'  # Name of the CSV file
MINIMUM_TWEETS = 400  # Modify this to the desired number of tweets to scrape

# Get the last tweet count from the CSV file
def get_last_tweet_count(filename):
    if os.path.exists(filename):
        with open(filename, 'r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            rows = list(reader)
            if len(rows) > 1:  # Ensure there's at least one row of data
                return int(rows[-1][0])  # The last tweet count from the last row
    return 0


# Create a new CSV file or append to an existing one
def write_tweet_to_csv(filename, tweet_data):
    file_exists = os.path.exists(filename)
    
    with open(filename, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # If file doesn't exist, write the header
        if not file_exists:
            writer.writerow(['Tweet_count', 'Username', 'Text', 'Created_At', 'Retweets', 'Likes', 'Views'])
        
        # Write the tweet data
        writer.writerow(tweet_data)


# Define an async function to retrieve tweets with the term "Web3"
async def search_web3_tweets():
    await client.login(
        auth_info_1=USERNAME,
        password=PASSWORD
    )
    
    print(f"{datetime.now()} - Logged in successfully!")

    last_tweet_count = get_last_tweet_count(CSV_FILENAME)  # Get the last tweet count
    tweet_count = last_tweet_count
    tweets = None

    while tweet_count < last_tweet_count + MINIMUM_TWEETS:
        delay = random.uniform(*DELAY_SECONDS)
        print(f'{datetime.now()} - Sleeping for {delay:.2f} seconds...')
        await asyncio.sleep(delay)
        
        try:
            if tweets is None:
                print(f"{datetime.now()} - Searching for Web3 tweets...")
                tweets = await client.search_tweet(QUERY, product='Top')
            else:
                print(f"{datetime.now()} - Getting next batch of tweets...")
                tweets = await tweets.next()

        except TooManyRequests:
            print(f"{datetime.now()} - Rate limit hit. Sleeping for {RATE_LIMIT_DELAY // 60} minutes...")
            await asyncio.sleep(RATE_LIMIT_DELAY)
            continue
        
        if not tweets:
            print(f"{datetime.now()} - No more tweets found.")
            break
        
        # Loop through the retrieved tweets and store the data
        for tweet in tweets:
            if any(keyword.lower() in tweet.text.lower() for keyword in WEB3_TOPICS):
                tweet_count += 1
                tweet_data = [
                    tweet_count,
                    tweet.user.name,
                    tweet.text,
                    tweet.created_at,
                    tweet.retweet_count,
                    tweet.favorite_count,
                    tweet.view_count if hasattr(tweet, 'view_count') else 'N/A'
                ]
                
                # Write the tweet data to CSV
                write_tweet_to_csv(CSV_FILENAME, tweet_data)
                
                # Output the tweet data to the console for feedback
                print(f"Tweet #{tweet_count}: {tweet_data}")

        if tweet_count >= last_tweet_count + MINIMUM_TWEETS:
            break

    print(f"{datetime.now()} - Done! Retrieved {tweet_count - last_tweet_count} new tweets.")

# Run the async function
asyncio.run(search_web3_tweets())
